In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
!pwd

/kaggle/working


## Titanic - Machine Learning from Disaster
### Start here! Predict survival on the Titanic and get familiar with ML basics

Dataset Description
-------------------

### Overview

The data has been split into two groups:

-   training set (train.csv)
-   test set (test.csv)

The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the "ground truth") for each passenger. Your model will be based on "features" like passengers' gender and class. You can also use [feature engineering ](https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/)to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

### Data Dictionary

| Variable | Definition | Key |
|---|---|---|
| survival | Survival | 0 = No, 1 = Yes |
| pclass | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| sex | Sex |  |
| Age | Age in years |  |
| sibsp | # of siblings / spouses aboard the Titanic |  |
| parch | # of parents / children aboard the Titanic |  |
| ticket | Ticket number |  |
| fare | Passenger fare |  |
| cabin | Cabin number |  |
| embarked | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

### Variable Notes

pclass: A proxy for socio-economic status (SES)\
1st = Upper\
2nd = Middle\
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...\
Sibling = brother, sister, stepbrother, stepsister\
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...\
Parent = mother, father\
Child = daughter, son, stepdaughter, stepson\
Some children travelled only with a nanny, therefore parch=0 for them.

### SOLUTION
---

# Importing Dataset

In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data  = pd.read_csv("/kaggle/input/titanic/test.csv")
test_ids = test_data["PassengerId"]

# Dataset Size and Shape

In [4]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  10692 	SHAPE :  (891, 12)
Test  -> SIZE :  4598 	SHAPE :  (418, 11)


# Dataset Exploration

In [5]:
train_data.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
383,384,1,1,"Holverson, Mrs. Alexander Oskar (Mary Aline To...",female,35.0,1,0,113789,52.0000,NaN,S
241,242,1,3,"Murphy, Miss. Katherine ""Kate""",female,NaN,1,0,367230,15.5000,NaN,Q
429,430,1,3,"Pickard, Mr. Berk (Berk Trembisky)",male,32.0,0,0,SOTON/O.Q. 392078,8.0500,E10,S
122,123,0,2,"Nasser, Mr. Nicholas",male,32.5,1,0,237736,30.0708,NaN,C
596,597,1,2,"Leitch, Miss. Jessie Wills",female,NaN,0,0,248727,33.0000,NaN,S
374,375,0,3,"Palsson, Miss. Stina Viola",female,3.0,3,1,349909,21.0750,NaN,S
135,136,0,2,"Richard, Mr. Emile",male,23.0,0,0,SC/PARIS 2133,15.0458,NaN,C
126,127,0,3,"McMahon, Mr. Martin",male,NaN,0,0,370372,7.7500,NaN,Q
285,286,0,3,"Stankovic, Mr. Ivan",male,33.0,0,0,349239,8.6625,NaN,C
42,43,0,3,"Kraeff, Mr. Theodor",male,NaN,0,0,349253,7.8958,NaN,C


In [6]:
test_data.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
28,920,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S
105,997,3,"Holthen, Mr. Johan Martin",male,28.0,0,0,C 4001,22.5250,NaN,S
179,1071,1,"Compton, Mrs. Alexander Taylor (Mary Eliza Ing...",female,64.0,0,2,PC 17756,83.1583,E45,C
130,1022,3,"Spinner, Mr. Henry John",male,32.0,0,0,STON/OQ. 369943,8.0500,NaN,S
259,1151,3,"Midtsjo, Mr. Karl Albert",male,21.0,0,0,345501,7.7750,NaN,S
115,1007,3,"Chronopoulos, Mr. Demetrios",male,18.0,1,0,2680,14.4542,NaN,C
287,1179,1,"Snyder, Mr. John Pillsbury",male,24.0,1,0,21228,82.2667,B45,S
94,986,1,"Birnbaum, Mr. Jakob",male,25.0,0,0,13905,26.0000,NaN,C
162,1054,2,"Wright, Miss. Marion",female,26.0,0,0,220844,13.5000,NaN,S
39,931,3,"Hee, Mr. Ling",male,NaN,0,0,1601,56.4958,NaN,S


# Cleaning Dataset

In [7]:
# Creating Function to Drop Columns from the DataFrame
# and Fill missing rows where data in not present

def dataset_cleaner(data):
    # Dropping Following columns as they will be unnecessary for predictions
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)

    # Filling missing Values
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    
    # U --> Unknown Token
    data.Embarked.fillna("U", inplace=True)

    return data


train_data = dataset_cleaner(train_data)
test_data  = dataset_cleaner(test_data)

In [8]:
print("Train -> SIZE : ", train_data.size, "\tSHAPE : ", train_data.shape)
print("Test  -> SIZE : ", test_data.size,  "\tSHAPE : ", test_data.shape)

Train -> SIZE :  7128 	SHAPE :  (891, 8)
Test  -> SIZE :  2926 	SHAPE :  (418, 7)


In [9]:
train_data.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
443,1,2,female,28.0,0,0,13.0000,S
795,0,2,male,39.0,0,0,13.0000,S
715,0,3,male,19.0,0,0,7.6500,S
368,1,3,female,28.0,0,0,7.7500,Q
272,1,2,female,41.0,0,1,19.5000,S
623,0,3,male,21.0,0,0,7.8542,S
880,1,2,female,25.0,0,1,26.0000,S
233,1,3,female,5.0,4,2,31.3875,S
478,0,3,male,22.0,0,0,7.5208,S
665,0,2,male,32.0,2,0,73.5000,S


In [10]:
test_data.sample(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
408,3,female,27.0,0,0,7.7208,Q
335,1,male,30.0,0,0,26.0000,S
91,3,male,27.0,0,0,7.7750,S
370,2,male,21.0,1,0,11.5000,S
126,3,male,22.0,0,0,7.7958,S
5,3,male,14.0,0,0,9.2250,S
81,1,male,67.0,1,0,221.7792,S
23,1,male,21.0,0,1,61.3792,C
314,1,female,55.0,0,0,135.6333,C
148,1,male,27.0,0,0,26.5500,S


In [11]:
from sklearn import preprocessing
lab_enc = preprocessing.LabelEncoder()

def encoder(data):
    # Encoding the following Columns values to encoded integers
    cols = ["Sex", "Embarked"]

    for col in cols:
        data[col] = lab_enc.fit_transform(data[col])
    

encoder(train_data)
encoder(test_data)        

# Training Model

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x = train_data.drop("Survived", axis=1)
y = train_data["Survived"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
classifier = LogisticRegression(random_state=3, max_iter=3000)
classifier = classifier.fit(x_train, y_train)

pred = classifier.predict(x_test)

In [14]:
from sklearn.metrics import accuracy_score

print("Classifier Accuracy :",accuracy_score(y_test, pred))

Classifier Accuracy : 0.8100558659217877


# Exporting Predictions

In [15]:
submit_pred = classifier.predict(test_data)

submit_df = pd.DataFrame({"PassengerId":test_ids.values,
                        "Survived": submit_pred})

submit_df.to_csv("submission.csv", index=False)
